# Dataset Sufficiency Analysis for Classification Tutorial


## _Problem Statement_

For machine learning tasks, often we would like to evaluate the performance of a model on a small, preliminary dataset. In situations where data collection is expensive, we would like to extrapolate hypothetical performance out to a larger dataset.

DataEval has introduced a method projecting performance via _sufficiency curves_.


### _When to use_

The `Sufficiency` class should be used when you would like to extrapolate hypothetical performance. For example, if you have a small dataset, and would like to know if it is worthwhile to collect more data.


### _What you will need_

1. A particular model architecture.
2. Metric(s) that we would like to evaluate.
3. A dataset of interest.
4. A python environment with the following packages installed:
   - `dataeval` or `dataeval[all]`
   - `tabulate`


### _Setting up_

Let's import the required libraries needed to set up a minimal working example


In [ ]:
try:
    import google.colab  # noqa: F401

    # specify the version of DataEval (==X.XX.X) for versions other than the latest
    %pip install -q dataeval
    !export LC_ALL="en_US.UTF-8"
    !export LD_LIBRARY_PATH="/usr/lib64-nvidia"
    !export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
    !ldconfig /usr/lib64-nvidia
except Exception:
    pass

%pip install -q tabulate

In [ ]:
import os
import random
from typing import Dict, Sequence, cast

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchmetrics
from tabulate import tabulate
from torch.utils.data import DataLoader, Dataset, Subset

from dataeval.utils.dataset.datasets import MNIST
from dataeval.workflows import Sufficiency

np.random.seed(0)
np.set_printoptions(formatter={"float": lambda x: f"{x:0.4f}"})
torch.manual_seed(0)
torch.set_float32_matmul_precision("high")
device = "cuda" if torch.cuda.is_available() else "cpu"
torch._dynamo.config.suppress_errors = True

random.seed(0)
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

## Load data and define functions


Load the MNIST data and create the training and test datasets.
For the purposes of this example, we will use subsets of the training (2000) and test (500) data.


In [ ]:
# Download the mnist dataset and preview the images
train_ds = MNIST(
    root="./data", train=True, download=True, size=2000, unit_interval=True, dtype=np.float32, channels="channels_first"
)
test_ds = MNIST(
    root="./data", train=False, download=True, size=500, unit_interval=True, dtype=np.float32, channels="channels_first"
)

In [ ]:
fig = plt.figure(figsize=(8, 3))

for lbl in range(10):
    i = (train_ds.targets == lbl).nonzero()[0][0]
    img = train_ds.data[i, 0, :, :]
    ax = fig.add_subplot(2, 5, lbl + 1)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.imshow(img, cmap="gray_r")

Next, we define the network architecture we will be using.


In [ ]:
# Define our network architecture
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(6400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Compile the model
model = torch.compile(Net().to(device))

# Type cast the model back to Net as torch.compile returns a Unknown
# Nothing internally changes from the cast; we are simply signaling the type
model = cast(Net, model)

Finally, we define our custom training and evaluation functions. Sufficiency requires that the evaluation function returns a dictionary of the results.


In [ ]:
def custom_train(model: nn.Module, dataset: Dataset, indices: Sequence[int]):
    # Defined only for this testing scenario
    criterion = torch.nn.CrossEntropyLoss().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    epochs = 10

    # Define the dataloader for training
    dataloader = DataLoader(Subset(dataset, indices), batch_size=16)

    for epoch in range(epochs):
        for batch in dataloader:
            # Load data/images to device
            X = torch.Tensor(batch[0]).to(device)
            # Load targets/labels to device
            y = torch.Tensor(batch[1]).to(device)
            # Zero out gradients
            optimizer.zero_grad()
            # Forward propagation
            outputs = model(X)
            # Compute loss
            loss = criterion(outputs, y)
            # Back prop
            loss.backward()
            # Update weights/parameters
            optimizer.step()

In [ ]:
def custom_eval(model: nn.Module, dataset: Dataset) -> Dict[str, float]:
    metric = torchmetrics.Accuracy(task="multiclass", num_classes=10).to(device)
    result = 0

    # Set model layers into evaluation mode
    model.eval()
    dataloader = DataLoader(dataset, batch_size=16)
    # Tell PyTorch to not track gradients, greatly speeds up processing
    with torch.no_grad():
        for batch in dataloader:
            # Load data/images to device
            X = torch.Tensor(batch[0]).to(device)
            # Load targets/labels to device
            y = torch.Tensor(batch[1]).to(device)
            preds = model(X)
            metric.update(preds, y)
        result = metric.compute().cpu()
    return {"Accuracy": result}

## Initialize sufficiency metric

Attach the custom training and evaluation functions to the Sufficiency metric and define the number of models to train in parallel (stability), as well as the number of steps along the learning curve to evaluate.


In [ ]:
# Instantiate sufficiency metric
suff = Sufficiency(
    model=model,
    train_ds=train_ds,
    test_ds=test_ds,
    train_fn=custom_train,
    eval_fn=custom_eval,
    runs=5,
    substeps=10,
)

## Evaluate Sufficiency

Now we can evaluate the metric to train the models and produce the learning curve.


In [ ]:
# Train & test model
output = suff.evaluate()

In [ ]:
# Print out sufficiency output in a table format
formatted = {"Steps": output.steps, **output.measures}
print(tabulate(formatted, headers=list(formatted), tablefmt="pretty"))

In [ ]:
# Print out projected output values
projection = output.project([1000, 2000, 4000])
projected = {"Steps": projection.steps, **projection.measures}
print(tabulate(projected, list(projected), tablefmt="pretty"))

In [ ]:
### TEST ASSERTION CELL ###
assert -0.015 < output.measures["Accuracy"][-1] - projection.measures["Accuracy"][-2] < 0.015

In [ ]:
# Plot the output using the convenience function
_ = output.plot()

## Results

Using this learning curve, we can project performance under much larger datasets (with the same model).


## Predicting sample requirements

We can also predict the amount of training samples required to achieve a desired accuracy.

Let's say we wanted to see how many samples are needed to hit 90%, 95% and 99% accuracy given the learning curve.


In [ ]:
# Initialize the array of desired accuracies
desired_accuracies = np.array([0.90, 0.95, 0.99])

# Evaluate the learning curve to infer the needed amount of training data
samples_needed = output.inv_project({"Accuracy": desired_accuracies})

In [ ]:
# Print the amount of needed data needed to achieve the accuracies of interest
for i, accuracy in enumerate(desired_accuracies):
    print(f"To achieve {int(accuracy*100)}% accuracy, {int(samples_needed['Accuracy'][i])} samples are needed.")

The projection shows that given the current model, hitting an accuracy of 99% is improbable.


In [ ]:
# Future BER tie in?

images, labels = [], []
for data in train_ds:
    images.append(np.array(data[0]))
    labels.append(data[1])

images = np.array(images)
labels = np.array(labels)

from dataeval.metrics.estimators import ber

ber_output = ber(images, labels)
np.round(1 - ber_output.ber_lower, 3) * 100